In [1]:
import pandas as pd
from IPython.display import HTML

base_url = 'https://jobs.aarp.org'

def display(df):
    df = df.copy()
    df['title'] = df[['title', 'link']].apply(
        lambda row: f'<a href="{base_url}{row.link}">{row.title}</a>',
        axis=1
    )
    return HTML(df.to_html(
        columns=['title', 'description', 'company', 'city', 'state'],
        index=False,
        escape=False
    ))

In [2]:
df = pd.read_json('jobs.json', 'records', convert_dates=['posted'])

# create timestamp for today, normalize to 00:00:00
today = pd.to_datetime('today').normalize()
# match timezone with datetimes in df so subtraction works
today = today.tz_localize(df['posted'].dt.tz)
# create 'age' column for days old
df['age'] = (today - df['posted']).dt.days

# split location into city and state
df[['city', 'state']] = df['location'].str.split(', ', 1, expand=True)

# clean up / organize dataframe
df = df[['title', 'company', 'description', 'city', 'state', 'age', 'link']]

df.head()

,title,company,description,city,state,age,link
0,Web Applications Engineer 3,Ursus,ABOUT THE COMPANY: Our client is the number o...,San Jose,CA,1,/job/web-applications-engineer-3-san-jose-ca-3...
1,Principal Systems Engineer,RTX,Are you a rock star engineer? Do you have the ...,Anaheim,CA,1,/job/principal-systems-engineer-anaheim-ca-60f...
2,Sr. Software Engineer/Integrator,Braxton Science & Technology Group,TITLE: Sr. Software Engineer/Integrator DIV...,Colorado Springs,CO,1,/job/sr-software-engineer-integrator-colorado-...
3,"Software Engineer, Mid",Booz Allen Hamilton,The Challenge: Are you looking for an opport...,Rome,NY,1,/job/software-engineer-mid-rome-ny-47d40403ad4...
4,Application Security Engineer,Esri,Overview: Join us in this critical role helpin...,Redlands,CA,1,/job/application-security-engineer-redlands-ca...


In [3]:
data = df['title'].str.lower().str.contains('data')
eng = df['title'].str.lower().str.contains('engineer')
senior = df['title'].str.lower().str.contains('senior') | df['title'].str.lower().str.contains('sr.')

In [4]:
check_out = df[data & eng & ~senior].sort_values(['age'], ascending=True)
check_out

,title,company,description,city,state,age,link
8,"Machine Learning Engineering, Data Science Eng...",Salesforce,"To get the best candidate experience, please c...",San Francisco,CA,1,/job/machine-learning-engineering-data-science...
162,Systems Engineer- Data Ingest 16-SE0212--Chant...,Probity,PLEASE NOTE: This position requires an ACTIV...,Chantilly,VA,1,/job/systems-engineer-data-ingest-16-se0212-ch...
156,Data Scientist/ML Engineer,PA Consulting Group,Become part of the team Do you crave a coll...,New York,NY,1,/job/data-scientist-ml-engineer-new-york-ny-12...
129,Data Scientist / Machine Learning- Engineer,HP,Data Scientist / Machine Learning- EngineerJob...,Houston,TX,1,/job/data-scientist-machine-learning-engineer-...
128,Principal Data Engineer,Lending Club,Current Employees of LendingClub: Please apply...,San Francisco,CA,1,/job/principal-data-engineer-san-francisco-ca-...
...,...,...,...,...,...,...,...
800,Principal Machine Learning Data Engineer (San ...,Okta,This is an opportunity to join our fast-growin...,New York,NY,4,/job/principal-machine-learning-data-engineer-...
728,"Platform Database Engineer, Digital Infrastruc...",Outfront Media,The Platform Database Engineer is responsible ...,New York,NY,4,/job/platform-database-engineer-digital-infras...
990,Data Center Operations Engineer,The Judge Group,"Apply Now Posted : November 20, 2020 Job Type ...",Wheat Ridge,CO,4,/job/data-center-operations-engineer-wheat-rid...
268,Database Engineer,Procter & Gamble,Information Technology (IT) at Procter & Gambl...,Cincinnati,OH,12,/job/database-engineer-cincinnati-oh-22e1af502...


In [5]:
display(check_out)

title,description,company,city,state
"Machine Learning Engineering, Data Science Engineering","To get the best candidate experience, please consider applying for a maximum of 3 roles within 12 months to ensure you are not duplicating efforts. Job Category Products and Technology Job Details Machine Learning Engineering and Data Science Engineering Roles We're hiring across tea...",Salesforce,San Francisco,CA
"Systems Engineer- Data Ingest 16-SE0212--Chantilly, VA","PLEASE NOTE: This position requires an ACTIVE Clearance Level of TS/SCI with Polygraph . To be considered for this position, you MUST have an ACTIVE Clearance Level of TS/SCI with Polygraph . Job Overview: Support loading a variety of structured and unstructured data and preparing...",Probity,Chantilly,VA
Data Scientist/ML Engineer,Become part of the team Do you crave a collaborative organization where your contributions will make a strong impact? Do you want to develop products in the latest cloud-based technologies building ecosystems rather than creating client-facing slides? Are you ready to roll up your sleeves and...,PA Consulting Group,New York,NY
Data Scientist / Machine Learning- Engineer,"Data Scientist / Machine Learning- EngineerJob Description:Hewlett Packard Enterprise (HPE) advances the way people live and work. We bring together curious minds to create breakthrough technology solutions, helping our customers make their mark on the world. At HPE Global IT we provide consulting,...",HP,Houston,TX
Principal Data Engineer,"Current Employees of LendingClub: Please apply via your internal Workday AccountLendingClub Corporation (NYSE: LC) is the parent company of LendingClub Bank, National Association, Member FDIC, and the only full-spectrum fintech marketplace bank. Members can gain access to a broad range of financial...",Lending Club,San Francisco,CA
"Data Engineer, Mid","Location: Arlington, Virginia, USAJob Number: R0091854Data Engineer, MidKey Role:Leverage expertise in structured and unstructured data to perform data engineering activities on some of the coolest cutting-edge projects in the industry and work with Big Data tools. Architect data systems stand up da...",Booz Allen Hamilton,Arlington,VA
Data Engineer,"Labcorp OfferingsRegular, full-time or part-time employees working 20 or more hours per week are eligible for comprehensive benefits. Click here to see benefits highlights.RequirementsLicense/Certification/Education: Normally requires a B.S. Degree in Computer Science w/6+ years of experience.Qualif...",LabCorp,Durham,NC
Production Data Operations and Automation Engineer,"Production Data Operations and Automation EngineerAs a Production Operations Engineer, you will work closely with a team of Data, Business Intelligence and Dev/Prod Ops engineers focused on supporting the automation of data pipes, platform infrastructure availability, and code-based data products an...",Splunk,San Jose,CA
ETL Production Support Engineer - Data Technology Platform Services,"Job DescriptionImportant Note: During the application process, ensure your contact information (email and phone number) is up to date and upload your current resume when submitting your application for consideration. To participate in some selection activities you will need to respond to an invitati...",Wells Fargo,New York,NY
Data Engineer,"Description:The Life Sciences Products team is looking for Sr. Data Engineers to help us build next generation data pipelines, data infrastructure and databases, and to develop data processing that is scalable, reliable and automated. The ideal candidate has demonstrated experience going from unders...",TEKsystems,San Francisco,CA
